In [21]:
import pandas as pd
import numpy as np
import datetime

# Replicating the 9 features from Online gambling's daily_aggregate dataset

Basically transforms the df_purchases to a daily aggregate, and from that daily aggregate (df_purchases_dailyaggregate.pkl), an analytic dataset (df_purchases_analytic.pkl) will be computed, with one row per user and 9 features.

`UserID	sum_stakes_fixedodds	sum_bets_fixedodds	bettingdays_fixedodds	duration_fixedodds	frequency_fixedodds	bets_per_day_fixedodds	euros_per_bet_fixedodds	net_loss_fixedodds	percent_lost_fixedodds`

In [22]:
df_purchases_value = pd.read_pickle('../processed_dataframes/df_purchases_value.pkl')
display(df_purchases_value.sample(4))
print(df_purchases_value.shape)

,datetime_zh,timestamp,user,src,src_en,src_type,src_value,out,out_1_nopar,out_1_par,out_2_nopar,out_2_par_1,out_2_par_2,out_3,out_type,out_value
1922416,2023-03-01 05:39:05,1677620345,SW***-KZPN,裂空武器箱,Fracture Case,Cases,2.5,MAG-7 | 北冥有鱼,MAG-7,,北冥有鱼,,,,Heavy,0.39
168398,2022-12-26 04:39:24,1672000764,S9***-ZMGQ,地平线武器箱,Horizon Case,Cases,2.5,MP9 | 毛细血管,MP9,,毛细血管,,,,SMGs,0.12
1287224,2023-02-05 18:04:12,1675591452,AA***-T8FL,命悬一线武器箱,Clutch Case,Cases,2.5,MP9 | 黑砂,MP9,,黑砂,,,,SMGs,0.04
938771,2023-01-23 18:05:43,1674468343,SY***-WPHQ,幻彩 3 号武器箱,Chroma 3 Case,Cases,2.5,M249 | 鬼影,M249,,鬼影,,,,Heavy,0.06


(1926179, 16)


## Preparing dataset

In [10]:
# Let's get rid of all non-relevant features for now
df_purchases_value['date'] = df_purchases_value['datetime_zh'].dt.floor('d') # keep just the date, not the time for each purchase
df_purchases_value = df_purchases_value[['date', 'user', 'src_value', 'out_value']]
display(df_purchases_value.sample(5))
print(df_purchases_value.shape)

,date,user,src_value,out_value
668007,2023-01-14,AX***-XNGQ,2.50,0.05
577921,2023-01-10,S8***-3MNA,2.50,0.16
713954,2023-01-15,AV***-2QRQ,0.95,0.03
1166800,2023-02-01,SK***-8WTJ,0.95,0.03
638998,2023-01-12,A6***-43QA,0.95,0.03


(1926179, 4)


In [4]:
# Remove null rows (we lose ~6.6% of the rows)
print(df_purchases_value.info())
print(df_purchases_value.shape)

# Convert out_value to float
df_purchases_value['out_value'] = df_purchases_value['out_value'].str.replace(' ', '') # A few values appear as '1 327.00' instead of '1327.00'
import numpy as np
df_purchases_value['out_value'] = df_purchases_value['out_value'].replace('', np.nan) # There could be empty strings too
df_purchases_value['out_value'] = df_purchases_value['out_value'].astype(float) # Convert out_value to float

df_purchases_value.dropna(subset=['src_value', 'out_value'], inplace=True)


print(df_purchases_value.info())
print(df_purchases_value.shape)

display(df_purchases_value[df_purchases_value['out_value'].isnull()])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1617059 entries, 0 to 1617058
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype         
---  ------     --------------    -----         
 0   date       1617059 non-null  datetime64[ns]
 1   user       1617059 non-null  object        
 2   src_value  1568484 non-null  float64       
 3   out_value  1579199 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 49.3+ MB
None
(1617059, 4)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1485798 entries, 0 to 1617056
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype         
---  ------     --------------    -----         
 0   date       1485798 non-null  datetime64[ns]
 1   user       1485798 non-null  object        
 2   src_value  1485798 non-null  float64       
 3   out_value  1485798 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 56.7+ MB
None
(1485798, 4)


,date,user,src_value,out_value


In [5]:
# We'll focus on the user `AA***-U3EE`
df_purchases_value[df_purchases_value['user'] == 'AA***-U3EE'].groupby(['date', 'user']).count().head(10)

,,src_value,out_value
date,user,,
2022-12-14,AA***-U3EE,23,23
2022-12-21,AA***-U3EE,15,15
2022-12-23,AA***-U3EE,41,41
2022-12-25,AA***-U3EE,18,18
2022-12-26,AA***-U3EE,117,117
2022-12-28,AA***-U3EE,56,56
2022-12-31,AA***-U3EE,98,98
2023-01-02,AA***-U3EE,95,95
2023-01-04,AA***-U3EE,34,34


## Daily aggregate dataset

In [6]:
# Calculating Turnover and Hold

def get_dailyaggregate(df_purchases_value):
    df_purchases_value['Turnover'] = df_purchases_value['src_value'] # all the money bet before any winnings are paid out or losses incurred. It's the same as the src_value then?
    df_purchases_value['Hold'] =  df_purchases_value['src_value']  - df_purchases_value['out_value'] # Total amount lost. Negative values mean the user won money.

    df_purchases_dailyaggregate = df_purchases_value.groupby(['date', 'user'], as_index=False).agg({'Turnover': 'sum', 'Hold': 'sum'})
    df_purchases_dailyaggregate['NumberofBets'] = df_purchases_value.groupby(['date', 'user']).size().reset_index(name='NumberofBets')['NumberofBets'] # Creates the column NumberofBets, by checking the size of the groupby of date and user.

    #display(df_purchases_dailyaggregate.head(10))
    return(df_purchases_dailyaggregate)

In [10]:
df_purchases_dailyaggregate = get_dailyaggregate(df_purchases_value)

display(df_purchases_dailyaggregate.tail(5))

,date,user,Turnover,Hold,NumberofBets
447105,2023-02-18,SZ***-U5EJ,2.50,2.42,1
447106,2023-02-18,SZ***-U7FL,2.50,2.44,1
447107,2023-02-18,SZ***-UVSJ,5.70,5.52,6
447108,2023-02-18,SZ***-YYHQ,3.00,2.97,1
447109,2023-02-18,SZ***-Z6PQ,4.75,4.60,5


In [11]:
# Save to pickle
df_purchases_dailyaggregate.to_pickle('../processed_dataframes/df_purchases_dailyaggregate.pkl')

## Analytic dataset

In [12]:
def get_analytic_dataset(df_purchases_dailyaggregate, datelimit=datetime.date.today()):
    # New empty dataframe
    df_purchases_analytic = pd.DataFrame()
    
    # Convert date feature to datetime
    df_purchases_dailyaggregate['date'] = pd.to_datetime(df_purchases_dailyaggregate['date'])
    
    # Only keep those values before the date limit
    df_purchases_dailyaggregate = df_purchases_dailyaggregate[df_purchases_dailyaggregate['date'] < datelimit]

    # 1. sum_stakes_fixedodds (sum of turnovers?)
    df_purchases_analytic['sum_stakes_fixedodds'] = df_purchases_dailyaggregate.groupby('user')['Turnover'].sum()

    # 2. sum_bets_fixedodds (sum of all the bets for all the days)
    df_purchases_analytic['sum_bets_fixedodds'] = df_purchases_dailyaggregate.groupby('user')['NumberofBets'].sum()

    # 3. bettingdays_fixedodds
    df_purchases_analytic['bettingdays_fixedodds'] = df_purchases_dailyaggregate.groupby('user')['NumberofBets'].count() # Make sure there are no days with users with 0 purchases

    # 4. duration_fixedodds (the number of days between the last and first date, both included)
    grouped = df_purchases_dailyaggregate.groupby('user')
    min_dates = grouped['date'].min()
    max_dates = grouped['date'].max()
    duration_fixedodds = (max_dates - min_dates).dt.days + 1
    df_purchases_analytic['duration_fixedodds'] = duration_fixedodds
    del min_dates, max_dates, grouped, duration_fixedodds

    # 5. frequency_fixedodds (percent of active days within the duration of gambling involvement) (bettingdays / duration)
    df_purchases_analytic['frequency_fixedodds'] = df_purchases_analytic['bettingdays_fixedodds'] / df_purchases_analytic['duration_fixedodds'] 

    # 6. bets_per_day_fixedodds
    df_purchases_analytic['bets_per_day_fixedodds'] = df_purchases_dailyaggregate.groupby('user')['NumberofBets'].mean()

    # 7. euros_per_bet_fixedodds (total monies wagered / total number of bets)
    df_purchases_analytic['euros_per_bet_fixedodds'] = df_purchases_analytic['sum_stakes_fixedodds'] / df_purchases_analytic['sum_bets_fixedodds']

    # 8. net_loss_fixedodds (sum of 'Hold')
    df_purchases_analytic['net_loss_fixedodds'] = df_purchases_dailyaggregate.groupby('user')['Hold'].sum()

    # 9. percent_lost_fixedodds (sum of 'Hold' / sum of 'Turnover')
    df_purchases_analytic['percent_lost_fixedodds'] = df_purchases_analytic['net_loss_fixedodds'] / df_purchases_analytic['sum_stakes_fixedodds']

    # Finally, reset index to the user is not the index.
    df_purchases_analytic = df_purchases_analytic.reset_index()

    return df_purchases_analytic

In [13]:
def get_sundays(start_date, end_date):
    # convert the start and end dates to datetime objects
    start_datetime = datetime.datetime.strptime(start_date, '%Y-%m-%d')
    end_datetime = datetime.datetime.strptime(end_date, '%Y-%m-%d')
    
    # calculate the first Sunday following the start date
    sunday = start_datetime + datetime.timedelta(days=(6 - start_datetime.weekday()))
    
    # create a list to store the Sunday dates
    sundays = []
    
    # loop through the Sundays between the start and end dates
    while sunday <= end_datetime:
        sundays.append(sunday.strftime('%Y-%m-%d'))
        sunday += datetime.timedelta(days=7)
    
    return sundays

In [16]:
def generate_weekly_analytics():
    # Generate an analytic dataset for each week until the present
    startdate = df_purchases_dailyaggregate['date'].min().strftime('%Y-%m-%d')
    enddate = df_purchases_dailyaggregate['date'].max().strftime('%Y-%m-%d')
    endweekdates = get_sundays(str(startdate), str(enddate))

    for date in endweekdates:
        print("Generating analytic dataset for ending date:", date)
        print(get_analytic_dataset(df_purchases_dailyaggregate, datelimit=date).shape)
        df_purchases_analytic = get_analytic_dataset(df_purchases_dailyaggregate, datelimit=date)
        df_purchases_analytic.to_pickle(f'../processed_dataframes/analytic/{date}_df_purchases_analytic.pkl')
        
        
generate_weekly_analytics()

Generating analytic dataset for ending date: 2022-12-18
(10004, 10)
Generating analytic dataset for ending date: 2022-12-25
(34359, 10)
Generating analytic dataset for ending date: 2023-01-01
(71373, 10)
Generating analytic dataset for ending date: 2023-01-08
(102025, 10)
Generating analytic dataset for ending date: 2023-01-15
(133448, 10)
Generating analytic dataset for ending date: 2023-01-22
(165591, 10)
Generating analytic dataset for ending date: 2023-01-29
(204155, 10)
Generating analytic dataset for ending date: 2023-02-05
(236431, 10)
Generating analytic dataset for ending date: 2023-02-12
(264699, 10)


In [17]:
# Maybe redundant? since we already have all the weekly dataframes
df_purchases_analytic.to_pickle('../processed_dataframes/df_purchases_analytic.pkl')